In [1]:
%load_ext autoreload
%autoreload 2
%aimport -jax
%aimport -jaxlib

In [2]:
import stanza.envs as envs
from jax.random import PRNGKey

env = envs.create("brax/ant")

In [3]:
state = env.reset(PRNGKey(42))

In [4]:
from stanza.util.ipython import display_html
display_html(env.render(state, mode="html"))


In [5]:
display_html(env.render(state, mode="html"))


In [6]:
import jax

from stanza.rl.nets import MLPActorCritic
from stanza.rl import EpisodicEnvironment

episode_env = EpisodicEnvironment(env, 1000)

net = MLPActorCritic(
    env.sample_action(PRNGKey(0))
)
params = net.init(PRNGKey(42),
    env.observe(env.sample_state(PRNGKey(0))))

In [11]:
from stanza.util.rich import StatisticsTable, ConsoleDisplay, LoopProgress
from stanza.rl.ppo import PPO
from stanza.train import Trainer
import optax

display = ConsoleDisplay()
display.add("ppo", StatisticsTable(), interval=1)
display.add("ppo", LoopProgress("RL"), interval=1)

ppo = PPO(
    episode_length=1000,
    total_timesteps=20_000_000,
    trainer = Trainer(
        optimizer=optax.chain(
            optax.clip_by_global_norm(0.5),
            optax.adam(3e-4, eps=1e-5)
        )
    )
)
with display as dh:
    trained_params = ppo.train(
        rng_key=PRNGKey(42),
        env=env,
        ac_apply=net.apply,
        init_params=params,
        rl_hooks=[dh.ppo]
    )

Output()

In [ ]:
from stanza import Partial
from stanza.rl import ACPolicy
import stanza.policies as policies

policy = ACPolicy(Partial(net.apply, trained_params.fn_params))

rollout = policies.rollout(env.step, env.reset(PRNGKey(7)), policy,
                           policy_rng_key=PRNGKey(42), 
                           model_rng_key=PRNGKey(69), 
                           observe=env.observe,
                           length=1000)

In [ ]:
display_html(env.render(rollout.states, mode="html"))


In [ ]:
display_html(env.render(state, mode="html"))


In [ ]:

display_html(env.render(state, mode="html"))